In [25]:
import pandas as pd
from PIL import Image
import json

In [24]:
bad_keys = ['name', 'type', 'image_quality']

In [12]:
df = pd.read_csv('Abdullah Khan i200894 CSV.csv')

In [13]:
df2 = pd.DataFrame(columns=['FileName','Text'])

In [38]:
def GetLabel2(i):
    dic = {}
    res = json.loads(df['region_attributes'][i])
    for k, val in res.items():
        dic[k] = val
    for key in dic.keys():
        if ((key not in bad_keys) and (dic[key] != '') and (dic[key] != 'MEDICINE NAME')):
            return dic[key]

In [16]:
def ShowSubImage(i):
    path = r"C:\Users\Shehroz\Desktop\University\DataInsight Lab\img"
    path = path + "\\" + df['filename'].iloc[i]
    im = Image.open(path)
    l = df['region_shape_attributes'].iloc[i]
    l = l.replace("{", "").replace("}", "").replace('"', '').split(",")
    l = l[1:]
    x = int(l[0].split(":")[1])
    y = int(l[1].split(":")[1])
    width = int(l[2].split(":")[1])
    height = int(l[3].split(":")[1])
    
    im1 = im.crop((x, y, x + width, y + height))
    im1.show()


In [7]:
def GetSubImage(i):
    path = r"C:\Users\Shehroz\Desktop\University\DataInsight Lab\img"
    path = path + "\\" + df['filename'].iloc[i]
    im = Image.open(path)
    l = df['region_shape_attributes'].iloc[i]
    l = l.replace("{", "").replace("}", "").replace('"', '').split(",")
    l = l[1:]
    x = int(l[0].split(":")[1])
    y = int(l[1].split(":")[1])
    width = int(l[2].split(":")[1])
    height = int(l[3].split(":")[1])
    
    im1 = im.crop((x, y, x + width, y + height))
    sub_image_name = 'img' + str(i) + '.png'
    im1.save(sub_image_name, 'PNG')
    return sub_image_name

In [40]:
for i in range(len(df)):
    l = [GetSubImage(i), GetLabel2(i)]
    df2.loc[len(df2)] = l


In [129]:
df2.to_csv('file2.csv', header=False, index=False)


TypeError: to_csv() got an unexpected keyword argument 'names'

In [6]:
df2 = pd.read_csv('file2.csv')
df2

FileNotFoundError: [Errno 2] No such file or directory: 'file2.csv'

In [131]:
df3

,File_name,Label
0,img0.png,Asif
1,img1.png,51
2,img2.png,Male
3,img3.png,31-7-22
4,img4.png,Multivitamin deficiency\nFever
...,...,...
1221,img1221.png,Fever
1222,img1222.png,ChestInfection
1223,img1223.png,120/80
1224,img1224.png,102


In [117]:
GetLabel(55)

'Cap'

In [ ]:
#Creating a new dataframe

for i in range(len(df)):
    

In [96]:
len(df)

1226

In [60]:
def GetLabel(l):
    

'{"name":"not_defined","type":"unknown","image_quality":{"good":true},"NAME":"","AGE":"","GENDER":"","DATE":"","BLOCK":"","MEDICINE TYPE":"","MEDICINE NAME":"","B":"","I":"","MEDICINE POWER":"","MEDICINE DOSE":"","B-U":"","I-U":"","DIAGNOSIS":"","HISTORY":"Cough\\nWeakness","BP":"","TEMP":"","WEIGHT":""}'

In [34]:
df['region_attributes'][6]

'{"name":"not_defined","type":"unknown","image_quality":{"good":true},"NAME":"","AGE":"","GENDER":"","DATE":"","BLOCK":"","MEDICINE TYPE":"","MEDICINE NAME":"","B":"","I":"","MEDICINE POWER":"","MEDICINE DOSE":"","B-U":"","I-U":"","DIAGNOSIS":"","HISTORY":"","BP":"120/80","TEMP":"","WEIGHT":""}'

In [19]:
df['region_shape_attributes'][0]["name"]

TypeError: string indices must be integers

In [15]:
df['region_attributes'][0]

'{"name":"not_defined","type":"unknown","image_quality":{"good":true},"NAME":"Asif","AGE":"","GENDER":"","DATE":"","BLOCK":"","MEDICINE TYPE":"","MEDICINE NAME":"","B":"","I":"","MEDICINE POWER":"","MEDICINE DOSE":"","B-U":"","I-U":"","DIAGNOSIS":"","HISTORY":"","BP":"","TEMP":"","WEIGHT":""}'

In [1]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import requests
from PIL import Image

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

# load image from the IAM dataset
url = "https://fki.tic.heia-fr.ch/static/img/a01-122-02.jpg"
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")

#Processor used for processing image into pixel values
pixel_values = processor(image, return_tensors="pt").pixel_values

generated_ids = model.generate(pixel_values)

generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [7]:
generated_text

'industry, " Mr. Brown commented icily. " Let us have a'

In [8]:
class IAMDataset(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=128):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text 
        file_name = self.df['file_name'][idx]
        text = self.df['text'][idx]
        # prepare image (i.e. resize + normalize)
        image = Image.open(self.root_dir + file_name).convert("RGB")
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(text, 
                                          padding="max_length", 
                                          max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

NameError: name 'Dataset' is not defined

In [5]:
from PIL import Image
from trocr.src.main import TrocrPredictor

ModuleNotFoundError: No module named 'trocr'

In [2]:
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

# load image from the IAM dataset
url = "https://fki.tic.heia-fr.ch/static/img/a01-122-02.jpg"
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")

pixel_values = processor(image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values)

generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

Downloading:   0%|          | 0.00/228 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.03k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [3]:
generated_text

'industry, " Mr. Brown commented icily. " Let us have a'